# Sting Jet Analysis

This notebook contains plots for sting jet analysis

In [2]:
# IMPORT MODULES

import xarray as xr # for netcdf
import cartopy.crs as ccrs # cartopy plots
import cartopy.feature as cfeature # for adding features to plots
import numpy as np # for arrays
import matplotlib.pyplot as plt # for plots

#%matplotlib inline

# Define a Cartopy 'ordinary' lat-lon coordinate reference system.
crs_latlon = ccrs.PlateCarree()

# Define a window for plotting
plonmin = -22
plonmax = -20
platmin = 48.5
platmax = 50.5
plev = 24

# Which resolution?
res='04'

# Integration period?
integr='6'

# NetCDF Files handles
f_APV = '/net/thermo/atmosdyn2/atroman/stingjet/tra_'+res+'/ATH20140212_02_'+integr+'h.nc'
f_P = '/net/thermo/atmosdyn2/atroman/stingjet/cdf_'+res+'/P20140212_02'
f_S = '/net/thermo/atmosdyn2/atroman/stingjet/cdf_'+res+'/S20140212_02'

# APV-Files
d_APV = xr.open_dataset(f_APV).sel(lat=slice(platmin,platmax),lon=slice(plonmin,plonmax),lev=plev) # use sel to only ready the lon / lat that you really need!!!
# Read coordinates
lon_APV, lat_APV = d_APV.lon.values, d_APV.lat.values
# Read data
TH_DIFF, ATHTOT, ATHRES, ATHLS, ATHLW, ATHCONV, ATHTURB = d_APV.TH_DIFF.values, d_APV.ATHTOT.values, d_APV.ATHRES.values, d_APV.ATHLS.values, d_APV.ATHLW.values, d_APV.ATHCONV.values, d_APV.ATHMIX.values

# P-Files
d_P = xr.open_dataset(f_P).sel(lat=slice(platmin,platmax),lon=slice(plonmin,plonmax),lev=plev)
lon_P, lat_P, SLP_P = d_P.lon.values, d_P.lat.values, d_P.SLP.values[0,0,:,:]

U = d_P.U.values[0,:,:]
V = d_P.V.values[0,:,:]

WIND = np.sqrt(U**2+V**2)

# S-Files
#d_S = xr.open_dataset(f_S)
#d_S

d_S = xr.open_dataset(f_S).sel(lat=slice(platmin,platmax),lon=slice(plonmin,plonmax),lev=plev)
THW = d_S.THW.values[0,:,:]

# Get an overview over dataset by simply typing
#data_APV

#print(APVTOT.shape)

# Generate figure
fig, axes = plt.subplots(2, 4, subplot_kw=dict(projection=ccrs.PlateCarree()))
#fig = plt.figure()

# Define figure size
fig.set_figheight(10)
fig.set_figwidth(18)

# Add projection and land-masks
#ax = plt.axes(projection=ccrs.PlateCarree())
#

# Plotting levels
THW_levels=np.arange(265,285,1)
APV_levels=np.arange(-7.5,7.5,0.1)

# THW Plot
filled_TH  = axes[0,0].contourf(lon_P,lat_P,THW,levels=THW_levels,cmap="Spectral_r",transform=crs_latlon) # who has a sexy PV colormap?
axes[0,0].set_title("THW [K]",size=20)

# PV-DIFF
filled_PV = axes[0,1].contourf(lon_APV,lat_APV,TH_DIFF[0,:,:],levels=APV_levels,cmap="RdBu_r",transform=crs_latlon)
axes[0,1].set_title("TH Difference",size=20)

# ATH TOT
filled_PV = axes[0,2].contourf(lon_APV,lat_APV,ATHTOT[0,:,:],levels=APV_levels,cmap="RdBu_r",transform=crs_latlon,extend="both")
axes[0,2].set_title("ATHTOT",size=20)

# RESID
filled_PV = axes[0,3].contourf(lon_APV,lat_APV,ATHRES[0,:,:],levels=APV_levels,cmap="RdBu_r",transform=crs_latlon,extend="both")
axes[0,3].set_title("ATHRES",size=20)

# ATHLS
filled_PV = axes[1,0].contourf(lon_APV,lat_APV,ATHLS[0,:,:],levels=APV_levels,cmap="RdBu_r",transform=crs_latlon,extend="both")
axes[1,0].set_title("ATHLS",size=20)

# ATHLW
filled_PV = axes[1,1].contourf(lon_APV,lat_APV,ATHLW[0,:,:],levels=APV_levels,cmap="RdBu_r",transform=crs_latlon,extend="both")
axes[1,1].set_title("ATHLWH",size=20)

# ATHCONVT
filled_PV = axes[1,2].contourf(lon_APV,lat_APV,ATHCONV[0,:,:],levels=APV_levels,cmap="RdBu_r",transform=crs_latlon,extend="both")
axes[1,2].set_title("ATHCONVT",size=20)

# ATHTURBT
filled_PV = axes[1,3].contourf(lon_APV,lat_APV,ATHTURB[0,:,:],levels=APV_levels,cmap="RdBu_r",transform=crs_latlon,extend="both")
axes[1,3].set_title("ATHTURBT",size=20)


# This is how I would like to add common stuff, but doesn not work whatsoever....
# Add comon stuff
#for ax in axes.flat:
    #ax.add_feature(cfeature.LAND,color="lightgray")
 #   ax.clabel(lines_wind,colors=['black'],manual=False,inline=True,fmt=' {:.1f} '.format)  
         
# Add common stuff        
for ax in axes.flat:
    ax.contour(lon_P,lat_P,WIND,levels=[53],colors="green",linewidths=3,transform=crs_latlon)
    #ax.clabel(lines_wind,colors=['black'],manual=False,inline=True,fmt=' {:.1f} '.format)
    lines_slp=ax.contour(lon_P,lat_P,SLP_P,levels=np.arange(960,1000,5),colors="black",transform=crs_latlon)
    ax.clabel(lines_slp,colors=['black'],manual=False,inline=True,fmt=' {:.1f} '.format)
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='LightGray', alpha=0.5, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_right = False
        
# Combined colorbar
#fig.colorbar(filled_PV, orientation='horizontal')
cbar=fig.colorbar(filled_PV, ax=axes.ravel().tolist(),orientation='horizontal',extend='both',fraction=0.046, pad=0.1)
cbar.set_label(label='Accumulated TH along trajectories (K '+integr+' h-1)', size=16, labelpad=-65)

# Make plot tidy
#fig.tight_layout(pad=3.0)

# Save figure to disk
fig.savefig('TH_plot_'+res+'_'+integr+'h_l'+str(plev)+'.png',bbox_inches='tight')

# Show figure
fig.show()


#lines_wind = axes[0,0].contour(lon_P,lat_P,WIND,levels=[50,55],colors="black",transform=crs_latlon)
#lines_slp = axes[0,0].contour(lon_P,lat_P,SLP_P,levels=np.arange(960,1000,5),cmap="gist_yarg_r",transform=crs_latlon)
#axes[0,0].clabel(lines_slp,colors=['black'],manual=False,inline=True,fmt=' {:.1f} '.format)


FileNotFoundError: [Errno 2] No such file or directory: b'/net/thermo/atmosdyn2/atroman/stingjet/tra_04/ATH20140212_02_6h.nc'